In [1]:
import pandas as pd
import numpy as np
import math
import helper
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR

from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut, ShuffleSplit, train_test_split, GridSearchCV 
from sklearn.feature_selection import SelectPercentile, SelectFromModel
import pickle    

# Подгрузка всех таблиц с данными по матчям

In [2]:
# дф с матчами и рейтингом каждой команды с патча 7.01 для Премиум
df_rating_teams_Premium = pd.read_csv('../tabel/table from Datdota/tabel with rating teams PREMIUM (with double Infamous).csv', index_col=0)
# удалить повторяющиеся матчи с Infamous, и сделать реиндекс 
df_rating_teams_Premium = df_rating_teams_Premium.drop(1).reset_index().drop('index', axis=1)

# ДФ с KDA героев для всех
df_KDA_hereos = pd.read_csv('../tabel/table from Datdota/tabel with KDA heroes in match.csv', index_col=0)

# ДФ c KDA героев для каждой команды индивидуально
df_KDA_hereos_teams = pd.read_csv('../tabel/table from Datdota/tabel with KDA heroes for team in match.csv', index_col=0)

# ДФ с контрпиками (head-to-head)
df_contrpick = pd.read_csv('../tabel/table from Datdota/tabel elo heroes in match vs enemies (head-to_head, contrPicks).csv', index_col=0)

# ДФ с типом героев
df_type_heroes = pd.read_csv('../tabel/table from Datdota/tabel summ features heroes PREMIUM (carry, support, nucker...).csv', index_col=0)

# ДФ с метовыми героями
df_meta_heroes = pd.read_csv('../tabel/table from Datdota/tabel with AvgElo heroes (meta).csv', index_col=0)

# ДФ с сигнатурками
df_signatures_heroes = pd.read_csv('../tabel/table from Datdota/tabel with AvgElo heroes for teams (signatures).csv', index_col=0)


# Создание основного ДФ для обучения и предсказания

In [3]:
# Соединение всех данных в один ДФ
main = pd.merge(df_rating_teams_Premium, df_KDA_hereos, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_KDA_hereos_teams, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_contrpick, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_type_heroes, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_meta_heroes, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_signatures_heroes, left_on='match_id', right_on='match_id',how='left')

# удаление не нужных колонок для обучения
main = main.drop(['match_id', 'start_time', 'radiant_team_id', 'radiant_score', 'dire_team_id',
           'dire_score', 'radiant_name', 'dire_name'], axis=1)
len(main)

3729

In [4]:
main.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3729 entries, 0 to 3728
Data columns (total 365 columns):
radiant_win                           3729 non-null bool
radiant_current_elo32                 3729 non-null float64
radiant_sevenDayAgo_elo32             3722 non-null float64
radiant_sevenDayAvg_elo32             3729 non-null float64
radiant_thirtyDayAgo_elo32            3685 non-null float64
radiant_thirtyDayAvg_elo32            3729 non-null float64
radiant_current_elo64                 3729 non-null float64
radiant_sevenDayAgo_elo64             3722 non-null float64
radiant_sevenDayAvg_elo64             3729 non-null float64
radiant_thirtyDayAgo_elo64            3685 non-null float64
radiant_thirtyDayAvg_elo64            3729 non-null float64
radiant_mu_glicko                     3729 non-null float64
radiant_rating_glicko                 3729 non-null float64
radiant_ratingSevenDaysAgo_glicko     3729 non-null float64
radiant_sigma_glicko                  3729 non-null flo

In [5]:
# Заполнить пустые значения на 0
main = main.fillna(0)

# Создание обучающих и тестовых данных

In [6]:
# удалть имена команд и сделать обучающие данные и ответы
X = main.drop(['radiant_win'], axis=1)
y = main['radiant_win']

# СДЕЛАТЬ 1 или 0 вместо true false
y = y.astype(int)

# деление на тестовые и обучающую выборку
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

# Решетчатый поиск на Gradient Bosting

In [8]:
# Праметры GB
# param_grid = {'n_estimators': [25, 50, 100, 150], 'max_depth': [10, 25, 50, 100],
#               'max_features': [50, 100, 200, 300]}
param_grid = {'n_estimators': [100], 'max_depth': [10],
              'max_features': [50]}
grid_search_GB = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, n_jobs=-1)

grid_search_GB.fit(X_train,y_train)

print("Правильность на тестовом наборе: {:.2f}".format(grid_search_GB.score(X_test, y_test)))
print("Наилучшие значения параметров: {}".format(grid_search_GB.best_params_))
print("Наилучшее значение кросс-валидац. правильности:{:.2f}".format(grid_search_GB.best_score_))

Правильность на тестовом наборе: 0.72
Наилучшие значения параметров: {'max_depth': 10, 'max_features': 50, 'n_estimators': 100}
Наилучшее значение кросс-валидац. правильности:0.73


In [16]:
import xgboost as xgb
# fit model no training data
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [17]:
model.score(X_test, y_test)

0.744908896034298

In [51]:
main.loc[1001, 'radiant_H1elo_vs_enemies':]

radiant_H1elo_vs_enemies    -15.3443
radiant_H2elo_vs_enemies    -7.53192
radiant_H3elo_vs_enemies    -2.70077
radiant_H4elo_vs_enemies     16.1597
radiant_H5elo_vs_enemies     2.75975
dire_H1elo_vs_enemies       -2.03825
dire_H2elo_vs_enemies        -14.792
dire_H3elo_vs_enemies       -6.54423
dire_H4elo_vs_enemies        4.15484
dire_H5elo_vs_enemies          9.947
radiant_Disabler                   4
radiant_Nuker                      4
radiant_Carry                      2
radiant_Initiator                  4
radiant_Escape                     2
radiant_Durable                    2
radiant_Support                    2
radiant_Pusher                     1
radiant_Jungler                    1
dire_Disabler                      5
dire_Nuker                         2
dire_Carry                         4
dire_Initiator                     3
dire_Escape                        2
dire_Durable                       1
dire_Support                       2
dire_Pusher                        1
d

In [55]:
df_meta_heroes[df_meta_heroes['match_id'] == 3274564960]

,radiant_H1_AvgElo,radiant_H2_AvgElo,radiant_H3_AvgElo,radiant_H4_AvgElo,radiant_H5_AvgElo,dire_H1_AvgElo,dire_H2_AvgElo,dire_H3_AvgElo,dire_H4_AvgElo,dire_H5_AvgElo,match_id
1007,0.0,0.0,5.022753,-7.223005,0.588956,-0.327158,-0.711979,0.560249,-1.010222,5.871091,3274564960
